# **Make the df that summarises codon chnage during codon usage bias mutl CDS sim** 

In [1]:
import json
import pandas as pd
from datetime import datetime

In [2]:
cds_names = open("data/redo_multDif_exons_2021_02_07/cdsNames_list.txt").readlines()
for i,cds in enumerate(cds_names): 
    cds_names[i]= cds.rstrip("\n")

model = json.load(open("../Human_mutability_model/Model_2020_12_02_genomeWide.txt"))

ntrial = 10 

In [3]:
#creating the empty dictionary
codon_change_dict = {}
for key in model.keys(): 
    codon_change_dict[key] = []
    
codon_final_count_dict = {}
for key in model.keys(): 
    codon_change_dict[key] = []

for n in range(1, ntrial+1): 
    #make the edict that counts all triplets for each trial 
    curTrial_codon_change_dict  = {}
    for key in codon_change_dict.keys(): 
        curTrial_codon_change_dict[key] = {"initial":0, "6x":0}
    
    #start counting the triplets across all cds used 
    for cds_name in cds_names: 
        #read in the intiial dna 
        cur_dna_gen0_filepath= "data/redo_multDif_exons_2021_02_07/CodingUsage_cds{c}_DNA_gen0.txt".format(c=cds_name)
        cur_dna_gen0_str = open(cur_dna_gen0_filepath).readlines()[0]
        #read in the 6x coverage dna 
        cur_dna_filepath = "data/redo_multDif_exons_2021_02_07/CodingUsage_cds{c}_Trial{n}_DNA_6x.txt".format(c=cds_name, n=n)
        cur_dna_str = open(cur_dna_filepath).readlines()[0]
        
        #codon initial count
        for i in range(1, len(cur_dna_gen0_str)-1, 3): #verified to count codons in a strign mutliple of 3 
            cur_triplet = cur_dna_gen0_str[i-1: i+2]
            curTrial_codon_change_dict[cur_triplet]["initial"]+=1
        #count the 6x dna 
        for i in range(1, len(cur_dna_str)-1, 3): 
            cur_triplet = cur_dna_str[i-1: i+2]
            curTrial_codon_change_dict[cur_triplet]["6x"]+=1
            
    #calculate the prop chnage and append to the codonchnage dict
    for key,value in curTrial_codon_change_dict.items():
        cur_prop_chnage = (value["6x"]-value["initial"])/value["initial"]
        codon_change_dict[key].append(cur_prop_chnage)

In [4]:
curTrial_codon_change_dict

{'TGC': {'initial': 414, '6x': 380},
 'GCG': {'initial': 567, '6x': 302},
 'CGG': {'initial': 327, '6x': 189},
 'AGA': {'initial': 341, '6x': 507},
 'GTT': {'initial': 494, '6x': 553},
 'TTG': {'initial': 534, '6x': 674},
 'GTG': {'initial': 507, '6x': 540},
 'ATA': {'initial': 521, '6x': 646},
 'TAA': {'initial': 8, '6x': 8},
 'TGA': {'initial': 8, '6x': 8},
 'ACC': {'initial': 473, '6x': 410},
 'AAT': {'initial': 754, '6x': 918},
 'ATT': {'initial': 523, '6x': 713},
 'TAC': {'initial': 403, '6x': 381},
 'AGC': {'initial': 496, '6x': 453},
 'CCG': {'initial': 516, '6x': 308},
 'CCT': {'initial': 492, '6x': 500},
 'AAG': {'initial': 1056, '6x': 1058},
 'TTC': {'initial': 547, '6x': 511},
 'CCC': {'initial': 534, '6x': 429},
 'TCA': {'initial': 550, '6x': 705},
 'GCT': {'initial': 566, '6x': 526},
 'ACT': {'initial': 463, '6x': 651},
 'GAC': {'initial': 901, '6x': 726},
 'ATC': {'initial': 517, '6x': 497},
 'ACA': {'initial': 533, '6x': 774},
 'ATG': {'initial': 775, '6x': 757},
 'GCC':

**building and saving the codon chnage df**

In [17]:
codon_change_df = pd.DataFrame.from_dict(codon_change_dict)
(codon_change_df.index) = ["Trial"+str(n) for n in range(1,ntrial+1)] #rename rows to trials

In [18]:
codon_change_df

,TGC,GCG,CGG,AGA,GTT,TTG,GTG,ATA,TAA,TGA,...,ACG,GAG,CTG,CTA,GAT,GTC,GGT,TCG,TAT,AAA
Trial1,-0.108696,-0.476190,-0.428135,0.510264,0.062753,0.205993,-0.001972,0.193858,0.0,0.0,...,-0.406742,-0.238849,0.020716,0.048736,-0.082609,-0.177024,-0.038306,-0.421933,0.230392,0.263810
Trial2,-0.118357,-0.477954,-0.425076,0.431085,0.054656,0.211610,0.055227,0.238004,0.0,0.0,...,-0.435955,-0.235971,-0.005650,-0.012635,-0.048913,-0.233522,-0.022177,-0.440520,0.227941,0.243810
Trial3,-0.125604,-0.488536,-0.425076,0.407625,0.056680,0.230337,0.005917,0.249520,0.0,0.0,...,-0.442697,-0.214388,0.009416,0.018051,-0.082609,-0.182674,-0.042339,-0.434944,0.247549,0.231429
Trial4,-0.091787,-0.492063,-0.422018,0.472141,0.082996,0.202247,-0.017751,0.259117,0.0,0.0,...,-0.431461,-0.233094,0.009416,0.032491,-0.047826,-0.197740,-0.038306,-0.425651,0.257353,0.248571
Trial5,-0.055556,-0.486772,-0.403670,0.337243,0.070850,0.217228,0.029586,0.214971,0.0,0.0,...,-0.449438,-0.238849,-0.015066,-0.009025,-0.092391,-0.177024,-0.020161,-0.436803,0.191176,0.260952
Trial6,-0.048309,-0.483245,-0.412844,0.445748,0.091093,0.181648,0.013807,0.274472,0.0,0.0,...,-0.433708,-0.224460,-0.011299,0.077617,-0.066304,-0.212806,-0.010081,-0.436803,0.294118,0.226667
Trial7,-0.086957,-0.483245,-0.412844,0.422287,0.119433,0.232210,-0.009862,0.184261,0.0,0.0,...,-0.404494,-0.230935,0.007533,0.018051,-0.088043,-0.197740,0.000000,-0.442379,0.296569,0.260952
Trial8,-0.154589,-0.488536,-0.431193,0.507331,0.157895,0.234082,0.057199,0.220729,0.0,0.0,...,-0.435955,-0.224460,-0.033898,0.045126,-0.068478,-0.186441,-0.044355,-0.429368,0.186275,0.254286
Trial9,-0.079710,-0.488536,-0.428135,0.466276,0.129555,0.250936,0.084813,0.205374,0.0,0.0,...,-0.406742,-0.227338,0.037665,0.021661,-0.094565,-0.216573,0.002016,-0.433086,0.208333,0.233333
Trial10,-0.082126,-0.467372,-0.422018,0.486804,0.119433,0.262172,0.065089,0.239923,0.0,0.0,...,-0.433708,-0.220144,-0.062147,0.032491,-0.055435,-0.173258,-0.014113,-0.438662,0.335784,0.226667


In [29]:
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
codon_change_df.to_csv("data/redo_multDif_exons_2021_02_07/{t}_codonUsage_propChange_multCds.csv".format(t=timestamp))